## Topic Modeling

In [1]:
from pyforest import *

In [2]:
data = pd.read_csv("abcnews-date-text.csv", error_bad_lines=False)
data.head()

<IPython.core.display.Javascript object>

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [3]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [4]:
print(f"Length of Documents: {len(documents)}")
documents[:5]

Length of Documents: 1186018


,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


# Data Pre-processing

We will perform the following steps:

- **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
- Words that have fewer than 3 characters are removed.
- All **stopwords** are removed.
- Words are **lemmatized** — words in third person are changed to first person and verbs in past and future tenses are changed into present.
- Words are **stemmed** — words are reduced to their root form.

In [30]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Ashish
[nltk_data]     Patel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmitize Example

In [38]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


### Stemmer Example

In [39]:
stemmer = SnowballStemmer('english')
original_words = [
    'caresses', 'flies', 'dies', 'mules', 'denied', 'died', 'agreed', 'owned',
    'humbled', 'sized', 'meeting', 'stating', 'siezing', 'itemization',
    'sensational', 'traditional', 'reference', 'colonizer', 'plotted'
]
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data={'original word': original_words, 'stemmed': singles})

<IPython.core.display.Javascript object>

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [45]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))


def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(
                token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [46]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

In [48]:
doc_sample = documents[documents['index'] == 4309].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [52]:
processed_docs = documents['headline_text'].map(preprocess)

In [53]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

### Bag of words on the dataset

In [54]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [55]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [56]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [58]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4309]

[(76, 1), (112, 1), (484, 1), (4038, 1)]

In [60]:
bow_doc_4310 = bow_corpus[4309]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(
        bow_doc_4310[i][0], dictionary[bow_doc_4310[i][0]],
        bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4038 ("dampen") appears 1 time.


### TF-IDF

In [61]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [62]:
corpus_tfidf = tfidf[bow_corpus]

In [63]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5850076620505259),
 (1, 0.38947256567331934),
 (2, 0.4997099083387053),
 (3, 0.5063271308533074)]


### Running LDA using Bag of Words

In [64]:
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                       num_topics=10,
                                       id2word=dictionary,
                                       passes=2,
                                       workers=2)

In [65]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"hous" + 0.022*"south" + 0.020*"north" + 0.017*"bushfir" + 0.016*"miss" + 0.013*"interview" + 0.012*"west" + 0.012*"hospit" + 0.011*"coast" + 0.010*"investig"
Topic: 1 
Words: 0.032*"kill" + 0.023*"shoot" + 0.021*"protest" + 0.021*"dead" + 0.020*"attack" + 0.020*"polic" + 0.014*"offic" + 0.014*"assault" + 0.011*"michael" + 0.011*"bank"
Topic: 2 
Words: 0.057*"australia" + 0.046*"australian" + 0.026*"world" + 0.018*"canberra" + 0.017*"test" + 0.013*"win" + 0.011*"final" + 0.011*"farm" + 0.010*"return" + 0.009*"beat"
Topic: 3 
Words: 0.030*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"death" + 0.024*"murder" + 0.020*"woman" + 0.020*"crash" + 0.017*"face" + 0.016*"alleg" + 0.013*"trial"
Topic: 4 
Words: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"climat" + 0.012*"concern" + 0.011*"flood" + 0.011*"save" + 0.011*"fear"
Topic: 5 
Words: 0.021*"market" + 0.020*"news" + 0.018*"women" + 0.018*"live" + 0.016*"tasmania" + 0.013*"

In [66]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf,
                                             num_topics=10,
                                             id2word=dictionary,
                                             passes=2,
                                             workers=4)

In [67]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.014*"market" + 0.011*"stori" + 0.009*"tuesday" + 0.009*"share" + 0.009*"friday" + 0.008*"australian" + 0.007*"peter" + 0.007*"dollar" + 0.006*"financ" + 0.006*"novemb"
Topic: 1 Word: 0.009*"michael" + 0.009*"wall" + 0.008*"street" + 0.007*"inquest" + 0.006*"june" + 0.006*"tree" + 0.006*"know" + 0.006*"hong" + 0.006*"john" + 0.006*"kong"
Topic: 2 Word: 0.009*"monday" + 0.009*"kill" + 0.009*"thursday" + 0.008*"grandstand" + 0.007*"insid" + 0.006*"syria" + 0.006*"quiz" + 0.006*"babi" + 0.006*"mother" + 0.005*"victorian"
Topic: 3 Word: 0.026*"news" + 0.013*"interview" + 0.011*"search" + 0.010*"miss" + 0.009*"hobart" + 0.009*"david" + 0.008*"beach" + 0.007*"speak" + 0.007*"busi" + 0.007*"mark"
Topic: 4 Word: 0.010*"world" + 0.010*"australia" + 0.009*"final" + 0.009*"queensland" + 0.008*"leagu" + 0.007*"scott" + 0.007*"test" + 0.007*"cricket" + 0.006*"beat" + 0.006*"rugbi"
Topic: 5 Word: 0.018*"polic" + 0.018*"charg" + 0.015*"murder" + 0.012*"woman" + 0.011*"alleg" + 0.010*"

In [68]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf,
                                             num_topics=10,
                                             id2word=dictionary,
                                             passes=2,
                                             workers=4)

In [69]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.016*"news" + 0.016*"rural" + 0.010*"market" + 0.010*"price" + 0.008*"busi" + 0.007*"nation" + 0.007*"scott" + 0.006*"share" + 0.006*"sport" + 0.006*"mental"
Topic: 1 Word: 0.012*"bushfir" + 0.010*"weather" + 0.008*"david" + 0.007*"northern" + 0.006*"queensland" + 0.006*"victoria" + 0.006*"april" + 0.005*"territori" + 0.005*"farm" + 0.005*"wild"
Topic: 2 Word: 0.010*"friday" + 0.010*"john" + 0.009*"violenc" + 0.008*"juli" + 0.007*"andrew" + 0.007*"august" + 0.006*"domest" + 0.006*"cancer" + 0.005*"flight" + 0.005*"patient"
Topic: 3 Word: 0.011*"final" + 0.010*"world" + 0.009*"australia" + 0.007*"leagu" + 0.007*"cricket" + 0.007*"win" + 0.007*"grandstand" + 0.006*"open" + 0.006*"financ" + 0.006*"australian"
Topic: 4 Word: 0.018*"countri" + 0.013*"hour" + 0.011*"royal" + 0.010*"commiss" + 0.009*"climat" + 0.007*"care" + 0.006*"zealand" + 0.006*"rugbi" + 0.006*"australia" + 0.006*"age"
Topic: 5 Word: 0.017*"interview" + 0.016*"crash" + 0.013*"drum" + 0.010*"polic" + 0.009*

## Classification of the topics
Performance evaluation by classifying sample document using LDA Bag of Words model

In [70]:
processed_docs[4309]

['rain', 'help', 'dampen', 'bushfir']

In [71]:
for index, score in sorted(lda_model[bow_corpus[4310]],
                           key=lambda tup: -1 * tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score,
                                             lda_model.print_topic(index, 10)))


Score: 0.45221951603889465	 
Topic: 0.020*"donald" + 0.014*"farmer" + 0.014*"nation" + 0.013*"time" + 0.013*"rural" + 0.013*"council" + 0.012*"indigen" + 0.011*"school" + 0.011*"commiss" + 0.011*"plan"

Score: 0.44774875044822693	 
Topic: 0.031*"govern" + 0.020*"warn" + 0.018*"feder" + 0.015*"countri" + 0.015*"fund" + 0.014*"claim" + 0.014*"life" + 0.012*"say" + 0.012*"stori" + 0.012*"health"

Score: 0.012506862170994282	 
Topic: 0.035*"elect" + 0.018*"water" + 0.017*"state" + 0.015*"tasmanian" + 0.012*"labor" + 0.011*"liber" + 0.011*"morrison" + 0.010*"parti" + 0.010*"leader" + 0.010*"campaign"

Score: 0.012503857724368572	 
Topic: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker" + 0.012*"climat" + 0.012*"concern" + 0.011*"flood" + 0.011*"save" + 0.011*"fear"

Score: 0.01250350009649992	 
Topic: 0.023*"hous" + 0.022*"south" + 0.020*"north" + 0.017*"bushfir" + 0.016*"miss" + 0.013*"interview" + 0.012*"west" + 0.012*"hospit" + 0.011*"coast" + 0.010*"investig

### Performance evaluation by classifying sample document using LDA TF-IDF model

In [72]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]],
                           key=lambda tup: -1 * tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(
        score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5840891003608704	 
Topic: 0.012*"stori" + 0.008*"michael" + 0.006*"council" + 0.006*"jam" + 0.006*"centr" + 0.006*"know" + 0.005*"govern" + 0.005*"money" + 0.005*"quiz" + 0.005*"ash"

Score: 0.1756652444601059	 
Topic: 0.012*"bushfir" + 0.010*"weather" + 0.008*"david" + 0.007*"northern" + 0.006*"queensland" + 0.006*"victoria" + 0.006*"april" + 0.005*"territori" + 0.005*"farm" + 0.005*"wild"

Score: 0.15267711877822876	 
Topic: 0.019*"charg" + 0.018*"murder" + 0.013*"court" + 0.013*"alleg" + 0.013*"polic" + 0.010*"woman" + 0.010*"jail" + 0.010*"sentenc" + 0.009*"guilti" + 0.009*"accus"

Score: 0.012512458488345146	 
Topic: 0.024*"trump" + 0.013*"elect" + 0.013*"donald" + 0.010*"govern" + 0.008*"labor" + 0.007*"wednesday" + 0.007*"say" + 0.007*"thursday" + 0.007*"liber" + 0.006*"feder"

Score: 0.012511294335126877	 
Topic: 0.016*"news" + 0.016*"rural" + 0.010*"market" + 0.010*"price" + 0.008*"busi" + 0.007*"nation" + 0.007*"scott" + 0.006*"share" + 0.006*"sport" + 0.006*"mental

### Testing model on unseen document

In [73]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1 * tup[1]):
    print("Score: {}\t Topic: {}".format(score,
                                         lda_model.print_topic(index, 5)))

Score: 0.48324039578437805	 Topic: 0.021*"market" + 0.020*"news" + 0.018*"women" + 0.018*"live" + 0.016*"tasmania"
Score: 0.21708554029464722	 Topic: 0.035*"elect" + 0.018*"water" + 0.017*"state" + 0.015*"tasmanian" + 0.012*"labor"
Score: 0.18293601274490356	 Topic: 0.030*"polic" + 0.029*"charg" + 0.026*"court" + 0.024*"death" + 0.024*"murder"
Score: 0.0166776180267334	 Topic: 0.019*"chang" + 0.019*"say" + 0.015*"speak" + 0.015*"power" + 0.013*"worker"
Score: 0.01667742058634758	 Topic: 0.031*"govern" + 0.020*"warn" + 0.018*"feder" + 0.015*"countri" + 0.015*"fund"
Score: 0.016676612198352814	 Topic: 0.023*"hous" + 0.022*"south" + 0.020*"north" + 0.017*"bushfir" + 0.016*"miss"
Score: 0.016676612198352814	 Topic: 0.032*"kill" + 0.023*"shoot" + 0.021*"protest" + 0.021*"dead" + 0.020*"attack"
Score: 0.016676612198352814	 Topic: 0.057*"australia" + 0.046*"australian" + 0.026*"world" + 0.018*"canberra" + 0.017*"test"
Score: 0.016676612198352814	 Topic: 0.020*"donald" + 0.014*"farmer" + 0.014